# QC-Py-10 - Risk Management et Portfolio Management

> **Position sizing, stop-loss, drawdown control et risk models QuantConnect**
> Duree: 90 minutes | Niveau: Intermediaire-Avance | Python + QuantConnect

---

## Objectifs d'Apprentissage

A la fin de ce notebook, vous serez capable de :

1. Implementer des methodes de **position sizing** (Fixed Fractional, Kelly Criterion)
2. Configurer differents types de **stop-loss** (fixe, ATR-based, trailing)
3. Controler le **portfolio heat** et les limites d'exposition
4. Implementer un **drawdown monitor** avec liquidation automatique
5. Utiliser le **volatility scaling** pour ajuster l'exposition dynamiquement
6. Integrer les **Risk Models** natifs de QuantConnect
7. Construire une **strategie complete** avec tous ces elements

## Prerequisites

- Notebooks QC-Py-01 a 09 completes
- Comprehension des indicateurs techniques (ATR, moyennes mobiles)
- Notions de base en gestion de portefeuille

## Structure du Notebook

| Partie | Sujet | Duree |
|--------|-------|-------|
| 1 | Position Sizing (Fixed Fractional, Kelly) | 25 min |
| 2 | Stop-Loss et Take-Profit | 25 min |
| 3 | Portfolio Heat et Exposure Limits | 20 min |
| 4 | Drawdown Control et Volatility Scaling | 20 min |
| 5 | Risk Models QuantConnect | 15 min |
| 6 | Strategie Complete avec Risk Management | 20 min |

---

## Introduction : Pourquoi le Risk Management ?

Le risk management est souvent considere comme la partie la plus importante du trading algorithmique. Une strategie avec un edge modeste mais un excellent risk management performera mieux qu'une strategie avec un fort edge mais un mauvais risk management.

### Les Piliers du Risk Management

| Pilier | Description | Objectif |
|--------|-------------|----------|
| **Position Sizing** | Combien investir par trade | Maximiser les gains ajustes au risque |
| **Stop-Loss** | Quand sortir d'une position perdante | Limiter les pertes par trade |
| **Portfolio Heat** | Risque total du portefeuille | Eviter la ruine en cas de choc |
| **Drawdown Control** | Surveiller les pertes cumulees | Preserver le capital |
| **Exposure Limits** | Limites par secteur/actif | Diversification forcee |

### La Regle d'Or

> "Never risk more than 1-2% of your capital on a single trade."
> - Van K. Tharp, *Trade Your Way to Financial Freedom*

Cette regle simple protege contre les series de pertes (losing streaks) qui sont inevitables meme avec une strategie profitable.

In [ ]:
# Imports QuantConnect
from AlgorithmImports import *

# Imports pour analyse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Configuration matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("Imports reussis")

---

## Partie 1 : Position Sizing (25 min)

Le position sizing repond a la question : "Combien investir dans ce trade ?"

### 1.1 Fixed Fractional (10 min)

La methode **Fixed Fractional** consiste a risquer un pourcentage fixe du capital sur chaque trade. C'est la methode la plus populaire car elle est simple et s'adapte automatiquement a la taille du compte.

#### Principe

1. Definir le pourcentage de risque par trade (ex: 1%)
2. Calculer le montant en dollars a risquer : `risk_amount = capital * risk_percent`
3. Calculer la distance au stop-loss : `stop_distance = |entry_price - stop_price|`
4. Calculer le nombre d'actions : `shares = risk_amount / stop_distance`

#### Formule

$$\text{Position Size} = \frac{\text{Capital} \times \text{Risk\%}}{|\text{Entry Price} - \text{Stop Price}|}$$

In [ ]:
def calculate_position_size_fixed_fractional(portfolio_value, risk_percent, entry_price, stop_price):
    """
    Calcule la taille de position selon la methode Fixed Fractional.
    
    Parameters:
    -----------
    portfolio_value : float
        Valeur totale du portefeuille
    risk_percent : float
        Pourcentage du capital a risquer (ex: 0.01 pour 1%)
    entry_price : float
        Prix d'entree prevu
    stop_price : float
        Prix du stop-loss
    
    Returns:
    --------
    int : Nombre d'actions a acheter
    """
    # Montant en dollars a risquer
    risk_amount = portfolio_value * risk_percent
    
    # Distance au stop
    stop_distance = abs(entry_price - stop_price)
    
    # Eviter division par zero
    if stop_distance == 0:
        return 0
    
    # Nombre d'actions
    shares = int(risk_amount / stop_distance)
    
    return shares

# Exemple pratique
portfolio = 100000  # $100,000
risk_pct = 0.01     # 1% de risque par trade
entry = 150.00      # Prix d'entree SPY
stop = 145.00       # Stop-loss a $145

shares = calculate_position_size_fixed_fractional(portfolio, risk_pct, entry, stop)

print("Exemple Fixed Fractional:")
print(f"  Portfolio: ${portfolio:,}")
print(f"  Risk percent: {risk_pct:.1%}")
print(f"  Risk amount: ${portfolio * risk_pct:,.0f}")
print(f"  Entry price: ${entry}")
print(f"  Stop price: ${stop}")
print(f"  Stop distance: ${entry - stop}")
print(f"  Position size: {shares} shares")
print(f"  Position value: ${shares * entry:,.0f} ({shares * entry / portfolio:.1%} du portfolio)")

In [ ]:
class FixedFractionalAlgorithm(QCAlgorithm):
    """
    Algorithme utilisant la methode Fixed Fractional pour le position sizing.
    Risque 1% du capital par trade avec stop-loss fixe a 5%.
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Actifs
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        
        # Indicateurs
        self.sma_fast = self.SMA(self.spy, 20)
        self.sma_slow = self.SMA(self.spy, 50)
        
        # Risk parameters
        self.risk_percent = 0.01  # 1% du capital par trade
        self.stop_percent = 0.05  # Stop-loss a 5% sous le prix d'entree
        
        # Tracking
        self.entry_price = None
        self.stop_price = None
        
        self.SetWarmup(50)
    
    def CalculatePositionSize(self, entry_price):
        """
        Calcule la taille de position avec la methode Fixed Fractional.
        """
        portfolio_value = self.Portfolio.TotalPortfolioValue
        risk_amount = portfolio_value * self.risk_percent
        stop_price = entry_price * (1 - self.stop_percent)
        stop_distance = entry_price - stop_price
        
        if stop_distance <= 0:
            return 0
        
        shares = int(risk_amount / stop_distance)
        return shares, stop_price
    
    def OnData(self, data):
        if self.IsWarmingUp:
            return
        
        if not data.ContainsKey(self.spy):
            return
        
        current_price = self.Securities[self.spy].Price
        
        # Signal d'achat: Golden Cross
        if not self.Portfolio[self.spy].Invested:
            if self.sma_fast.Current.Value > self.sma_slow.Current.Value:
                # Calculer position size
                shares, stop_price = self.CalculatePositionSize(current_price)
                
                if shares > 0:
                    self.MarketOrder(self.spy, shares)
                    self.entry_price = current_price
                    self.stop_price = stop_price
                    
                    self.Debug(f"{self.Time.date()}: BUY {shares} shares @ ${current_price:.2f}")
                    self.Debug(f"  Stop-loss: ${stop_price:.2f}")
                    self.Debug(f"  Risk: ${shares * (current_price - stop_price):.0f} ({self.risk_percent:.1%})")
        
        # Gestion position existante
        else:
            # Stop-loss
            if current_price <= self.stop_price:
                self.Liquidate(self.spy)
                self.Debug(f"{self.Time.date()}: STOP-LOSS triggered @ ${current_price:.2f}")
                self.entry_price = None
                self.stop_price = None
            
            # Signal de vente: Death Cross
            elif self.sma_fast.Current.Value < self.sma_slow.Current.Value:
                self.Liquidate(self.spy)
                profit = current_price - self.entry_price
                self.Debug(f"{self.Time.date()}: SELL @ ${current_price:.2f} (P/L: ${profit:.2f}/share)")
                self.entry_price = None
                self.stop_price = None

print("FixedFractionalAlgorithm defini")
print("\nParametres:")
print("  - Risque par trade: 1%")
print("  - Stop-loss: 5% sous l'entree")

### Avantages du Fixed Fractional

| Avantage | Description |
|----------|-------------|
| **Simplicite** | Facile a implementer et comprendre |
| **Adaptabilite** | La taille de position s'adapte automatiquement au capital |
| **Protection** | Limite les pertes meme apres une serie de trades perdants |
| **Scalabilite** | Fonctionne aussi bien avec $10k qu'avec $1M |

> **Note pratique** : La plupart des traders professionnels utilisent 0.5% a 2% de risque par trade. Les traders agressifs montent jusqu'a 5%, mais cela augmente significativement le risque de ruine.

### 1.2 Kelly Criterion (15 min)

Le **Kelly Criterion** est une formule mathematique qui determine la taille de position optimale pour maximiser la croissance a long terme du capital.

#### Formule de Kelly

$$f^* = W - \frac{1-W}{R}$$

Ou :
- $f^*$ = fraction optimale du capital a investir
- $W$ = win rate (probabilite de gain)
- $R$ = ratio gain moyen / perte moyenne

#### Exemple

Si votre strategie a :
- Win rate = 55%
- Gain moyen = $200
- Perte moyenne = $100

Alors : $f^* = 0.55 - \frac{1-0.55}{200/100} = 0.55 - 0.225 = 0.325$ (32.5% du capital)

#### Important : Half-Kelly

En pratique, on utilise souvent **Half-Kelly** (50% de la valeur calculee) car :
- Les estimations de win rate et payoff sont incertaines
- Kelly assume une distribution normale (ce qui est rarement vrai)
- Le full Kelly peut etre trop agressif et volatile

In [ ]:
def kelly_fraction(win_rate, avg_win, avg_loss):
    """
    Calcule la fraction de Kelly optimale.
    
    Parameters:
    -----------
    win_rate : float
        Probabilite de gain (ex: 0.55 pour 55%)
    avg_win : float
        Gain moyen par trade gagnant (valeur positive)
    avg_loss : float
        Perte moyenne par trade perdant (valeur negative ou positive)
    
    Returns:
    --------
    float : Fraction optimale du capital (0 si negative)
    """
    # S'assurer que avg_loss est positif pour le calcul
    avg_loss = abs(avg_loss)
    
    if avg_loss == 0:
        return 0
    
    # Ratio reward/risk
    R = avg_win / avg_loss
    
    # Formule de Kelly
    kelly = win_rate - (1 - win_rate) / R
    
    # Ne jamais retourner une valeur negative (pas de short)
    return max(0, kelly)

# Exemples avec differents parametres
print("Exemples Kelly Criterion:")
print("=" * 60)

scenarios = [
    {"name": "Strategie moyenne", "win_rate": 0.55, "avg_win": 200, "avg_loss": 100},
    {"name": "Strategie haute win rate", "win_rate": 0.70, "avg_win": 100, "avg_loss": 150},
    {"name": "Strategie grand R", "win_rate": 0.35, "avg_win": 500, "avg_loss": 100},
    {"name": "Strategie pauvre", "win_rate": 0.45, "avg_win": 100, "avg_loss": 120},
]

for s in scenarios:
    full_kelly = kelly_fraction(s["win_rate"], s["avg_win"], s["avg_loss"])
    half_kelly = full_kelly / 2
    R = s["avg_win"] / s["avg_loss"]
    
    print(f"\n{s['name']}:")
    print(f"  Win Rate: {s['win_rate']:.0%}")
    print(f"  Avg Win: ${s['avg_win']}, Avg Loss: ${s['avg_loss']} (R = {R:.2f})")
    print(f"  Full Kelly: {full_kelly:.1%}")
    print(f"  Half Kelly: {half_kelly:.1%}")

In [ ]:
# Visualisation de l'effet du Kelly sur la croissance du capital
np.random.seed(42)

# Parametres de la strategie simulee
win_rate = 0.55
avg_win = 200
avg_loss = 100
n_trades = 100

# Calculer Kelly
full_kelly = kelly_fraction(win_rate, avg_win, avg_loss)
half_kelly = full_kelly / 2
quarter_kelly = full_kelly / 4

# Simuler les trades
wins = np.random.random(n_trades) < win_rate
returns = np.where(wins, avg_win, -avg_loss)

# Simuler la croissance pour differentes fractions
def simulate_growth(returns, fraction, initial_capital=10000):
    capital = [initial_capital]
    for r in returns:
        pct_return = r / initial_capital  # Normaliser
        new_capital = capital[-1] * (1 + fraction * pct_return / 100)
        capital.append(max(0, new_capital))  # Pas de capital negatif
    return capital

fractions = {
    "2x Kelly (over-betting)": full_kelly * 2,
    "Full Kelly": full_kelly,
    "Half Kelly": half_kelly,
    "Quarter Kelly": quarter_kelly,
    "Fixed 5%": 0.05,
}

# Graphique
plt.figure(figsize=(12, 6))

colors = ['red', 'orange', 'green', 'blue', 'purple']
for (name, frac), color in zip(fractions.items(), colors):
    growth = simulate_growth(returns, frac)
    plt.plot(growth, label=f"{name} ({frac:.1%})", color=color, linewidth=2)

plt.title("Impact du Position Sizing sur la Croissance du Capital\n(100 trades simules)", 
          fontsize=14, fontweight='bold')
plt.xlabel("Nombre de trades", fontsize=12)
plt.ylabel("Capital ($)", fontsize=12)
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nParametres: Win Rate = {win_rate:.0%}, Avg Win = ${avg_win}, Avg Loss = ${avg_loss}")
print(f"Kelly optimal: {full_kelly:.1%}")
print("\nObservation: Over-betting (2x Kelly) peut mener a la ruine meme avec un edge positif.")

### Quand utiliser Kelly vs Fixed Fractional ?

| Methode | Cas d'usage | Risques |
|---------|-------------|---------|
| **Fixed Fractional** | Trading discretionnaire, strategie non-statistiquement validee | Sous-optimal si edge connu |
| **Full Kelly** | Edge statistiquement prouve sur long historique | Volatilite elevee, drawdowns importants |
| **Half Kelly** | Compromis recommande, edge estime mais incertain | Meilleur risk-adjusted return |
| **Quarter Kelly** | Conservateur, debut de strategie | Croissance plus lente |

> **Recommandation** : Commencez avec Fixed Fractional ou Quarter Kelly, puis migrez vers Half Kelly une fois que vous avez au moins 50-100 trades avec des statistiques stables.

In [ ]:
class KellyPositionSizingAlgorithm(QCAlgorithm):
    """
    Algorithme utilisant le Kelly Criterion pour le position sizing.
    Calcule dynamiquement la fraction de Kelly basee sur l'historique des trades.
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        
        # Indicateurs
        self.rsi = self.RSI(self.spy, 14)
        self.atr = self.ATR(self.spy, 14)
        
        # Historique des trades pour calculer Kelly
        self.trade_results = []  # Liste de (win/loss, amount)
        self.min_trades_for_kelly = 20  # Minimum de trades avant d'utiliser Kelly
        
        # Position tracking
        self.entry_price = None
        
        # Risk parameters
        self.max_kelly = 0.25  # Plafonner Kelly a 25% max
        self.kelly_multiplier = 0.5  # Half-Kelly
        self.default_position_size = 0.10  # 10% avant d'avoir assez de trades
        
        self.SetWarmup(30)
    
    def KellyFraction(self):
        """
        Calcule la fraction de Kelly basee sur l'historique des trades.
        """
        if len(self.trade_results) < self.min_trades_for_kelly:
            return self.default_position_size
        
        # Separer wins et losses
        wins = [t[1] for t in self.trade_results if t[0]]
        losses = [abs(t[1]) for t in self.trade_results if not t[0]]
        
        if len(wins) == 0 or len(losses) == 0:
            return self.default_position_size
        
        # Calculer statistiques
        win_rate = len(wins) / len(self.trade_results)
        avg_win = np.mean(wins)
        avg_loss = np.mean(losses)
        
        if avg_loss == 0:
            return self.default_position_size
        
        # Formule de Kelly
        R = avg_win / avg_loss
        kelly = win_rate - (1 - win_rate) / R
        
        # Appliquer le multiplicateur et plafonner
        kelly = max(0, kelly) * self.kelly_multiplier
        kelly = min(kelly, self.max_kelly)
        
        return kelly
    
    def OnData(self, data):
        if self.IsWarmingUp:
            return
        
        if not data.ContainsKey(self.spy):
            return
        
        current_price = self.Securities[self.spy].Price
        
        # Signal d'achat: RSI < 30 (oversold)
        if not self.Portfolio[self.spy].Invested:
            if self.rsi.Current.Value < 30:
                # Calculer position size avec Kelly
                kelly = self.KellyFraction()
                self.SetHoldings(self.spy, kelly)
                self.entry_price = current_price
                
                self.Debug(f"{self.Time.date()}: BUY @ ${current_price:.2f}")
                self.Debug(f"  Position size: {kelly:.1%} (Kelly based on {len(self.trade_results)} trades)")
        
        # Signal de vente: RSI > 70 (overbought)
        else:
            if self.rsi.Current.Value > 70:
                # Enregistrer le resultat du trade
                pnl = current_price - self.entry_price
                is_win = pnl > 0
                self.trade_results.append((is_win, pnl))
                
                self.Liquidate(self.spy)
                
                self.Debug(f"{self.Time.date()}: SELL @ ${current_price:.2f} (P/L: ${pnl:.2f})")
                self.entry_price = None
    
    def OnEndOfAlgorithm(self):
        """Affiche les statistiques finales."""
        if len(self.trade_results) > 0:
            wins = sum(1 for t in self.trade_results if t[0])
            win_rate = wins / len(self.trade_results)
            
            self.Debug(f"\n=== STATISTIQUES FINALES ===")
            self.Debug(f"Total trades: {len(self.trade_results)}")
            self.Debug(f"Win rate: {win_rate:.1%}")
            self.Debug(f"Final Kelly: {self.KellyFraction():.1%}")

print("KellyPositionSizingAlgorithm defini")
print("\nCaracteristiques:")
print("  - Utilise Half-Kelly pour reduire la volatilite")
print("  - Plafonne a 25% du capital")
print("  - Necessite 20 trades minimum pour calcul dynamique")

---

## Partie 2 : Stop-Loss et Take-Profit (25 min)

Les stop-loss sont essentiels pour limiter les pertes. Il existe plusieurs types de stop-loss, chacun avec ses avantages.

### 2.1 Types de Stop-Loss (15 min)

| Type | Description | Avantages | Inconvenients |
|------|-------------|-----------|---------------|
| **Fixed %** | Stop a X% sous l'entree | Simple, previsible | Ne s'adapte pas a la volatilite |
| **ATR-based** | Stop a X * ATR | S'adapte a la volatilite | Plus complexe |
| **Support/Resistance** | Stop sous support technique | Techniquement solide | Necessite analyse |
| **Trailing** | Stop qui monte avec le prix | Protege les gains | Peut sortir trop tot |

In [ ]:
class StopLossTypesAlgorithm(QCAlgorithm):
    """
    Demonstration des differents types de stop-loss.
    """
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        
        # Indicateurs pour stop-loss
        self.atr = self.ATR(self.spy, 14)  # Average True Range
        self.sma = self.SMA(self.spy, 20)  # Support dynamique
        
        # Configuration stop-loss
        self.stop_type = "atr"  # Options: "fixed", "atr", "support"
        self.fixed_stop_percent = 0.05   # 5% pour fixed stop
        self.atr_multiplier = 2.0        # 2x ATR pour ATR stop
        
        # Tracking
        self.entry_price = None
        self.stop_price = None
        
        self.SetWarmup(20)
    
    def CalculateStopPrice(self, entry_price):
        """
        Calcule le prix de stop selon le type configure.
        """
        if self.stop_type == "fixed":
            # Stop fixe a X% sous l'entree
            return entry_price * (1 - self.fixed_stop_percent)
        
        elif self.stop_type == "atr":
            # Stop a X * ATR sous l'entree
            if not self.atr.IsReady:
                return entry_price * 0.95  # Fallback
            atr_value = self.atr.Current.Value
            stop_distance = self.atr_multiplier * atr_value
            return entry_price - stop_distance
        
        elif self.stop_type == "support":
            # Stop sous la SMA (support dynamique)
            if not self.sma.IsReady:
                return entry_price * 0.95  # Fallback
            return self.sma.Current.Value * 0.98  # 2% sous la SMA
        
        return entry_price * 0.95  # Default
    
    def OnData(self, data):
        if self.IsWarmingUp:
            return
        
        if not data.ContainsKey(self.spy):
            return
        
        current_price = self.Securities[self.spy].Price
        
        # Entry logic (simplifie)
        if not self.Portfolio[self.spy].Invested:
            # Acheter si prix > SMA
            if current_price > self.sma.Current.Value:
                self.entry_price = current_price
                self.stop_price = self.CalculateStopPrice(current_price)
                
                self.SetHoldings(self.spy, 0.95)
                
                self.Debug(f"{self.Time.date()}: BUY @ ${current_price:.2f}")
                self.Debug(f"  Stop type: {self.stop_type}")
                self.Debug(f"  Stop price: ${self.stop_price:.2f}")
                self.Debug(f"  Risk: {(current_price - self.stop_price) / current_price:.1%}")
        
        # Stop-loss check
        else:
            if current_price <= self.stop_price:
                self.Liquidate(self.spy)
                loss = (self.stop_price - self.entry_price) / self.entry_price
                self.Debug(f"{self.Time.date()}: STOP-LOSS @ ${current_price:.2f} (Loss: {loss:.1%})")
                self.entry_price = None
                self.stop_price = None

print("StopLossTypesAlgorithm defini")
print("\nTypes de stop disponibles:")
print("  - 'fixed': Stop fixe a 5% sous l'entree")
print("  - 'atr': Stop a 2x ATR sous l'entree (adaptatif)")
print("  - 'support': Stop sous la SMA 20 (support dynamique)")

In [ ]:
# Comparaison visuelle des types de stop-loss
np.random.seed(42)

# Simuler un prix avec tendance et volatilite
n_days = 60
dates = pd.date_range('2023-01-01', periods=n_days, freq='B')

# Prix simule (monte puis descend)
base_price = 100
trend = np.concatenate([
    np.linspace(0, 15, 30),   # Hausse
    np.linspace(15, 5, 30),   # Baisse
])
noise = np.random.normal(0, 2, n_days)
prices = base_price + trend + noise

# Entree au jour 5
entry_day = 5
entry_price = prices[entry_day]

# Calculer les differents stops
fixed_stop = entry_price * 0.95
atr_estimate = np.std(np.diff(prices[:20])) * 1.5  # Estimation ATR
atr_stop = entry_price - 2 * atr_estimate

# SMA pour support stop
sma_20 = pd.Series(prices).rolling(20).mean()
support_stop = sma_20 * 0.98

# Graphique
fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Subplot 1: Prix et stops
axes[0].plot(dates, prices, 'b-', linewidth=2, label='Prix')
axes[0].axhline(y=fixed_stop, color='red', linestyle='--', label=f'Fixed Stop (5%): ${fixed_stop:.2f}')
axes[0].axhline(y=atr_stop, color='orange', linestyle='--', label=f'ATR Stop (2x): ${atr_stop:.2f}')
axes[0].plot(dates, support_stop, color='green', linestyle='--', label='Support Stop (SMA-2%)')
axes[0].axvline(x=dates[entry_day], color='blue', linestyle=':', alpha=0.5)
axes[0].scatter([dates[entry_day]], [entry_price], color='blue', s=100, zorder=5, label=f'Entry: ${entry_price:.2f}')

axes[0].set_title('Comparaison des Types de Stop-Loss', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Prix ($)', fontsize=12)
axes[0].legend(loc='upper left')
axes[0].grid(True, alpha=0.3)

# Subplot 2: Distance au stop en %
fixed_distance = (prices - fixed_stop) / prices * 100
atr_distance = (prices - atr_stop) / prices * 100
support_distance = (prices - support_stop) / prices * 100

axes[1].fill_between(dates, 0, fixed_distance, alpha=0.3, color='red', label='Fixed Stop')
axes[1].fill_between(dates, 0, atr_distance, alpha=0.3, color='orange', label='ATR Stop')
axes[1].plot(dates, support_distance, color='green', linewidth=2, label='Support Stop')
axes[1].axhline(y=0, color='black', linewidth=2)

axes[1].set_title('Distance au Stop (%)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Date', fontsize=12)
axes[1].set_ylabel('Distance (%)', fontsize=12)
axes[1].legend(loc='upper right')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nObservations:")
print("  - Fixed stop: Ne s'adapte pas a la volatilite")
print("  - ATR stop: Plus large quand la volatilite augmente")
print("  - Support stop: Suit la tendance (SMA)")

### 2.2 Trailing Stop (10 min)

Le **Trailing Stop** est un stop qui se deplace avec le prix mais uniquement dans la direction favorable. Il protege les gains tout en laissant courir les tendances.

#### Principe

1. Definir une distance de trail (ex: 5% ou 2x ATR)
2. Quand le prix monte, le stop monte aussi
3. Quand le prix baisse, le stop reste fixe
4. Si le prix touche le stop, on sort

In [ ]:
class TrailingStopAlgorithm(QCAlgorithm):
    """
    Algorithme avec Trailing Stop dynamique.
    Le stop monte avec le prix mais ne descend jamais.
    """
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        
        # Indicateurs
        self.sma = self.SMA(self.spy, 50)
        self.atr = self.ATR(self.spy, 14)
        
        # Trailing stop parameters
        self.trailing_percent = 0.05      # 5% trailing stop
        self.use_atr_trailing = True      # Utiliser ATR au lieu de %
        self.atr_trailing_mult = 2.0      # 2x ATR pour le trail
        
        # Tracking
        self.entry_price = None
        self.highest_price = None
        self.current_stop = None
        
        self.SetWarmup(50)
    
    def UpdateTrailingStop(self, current_price):
        """
        Met a jour le trailing stop si le prix a atteint un nouveau sommet.
        """
        if current_price > self.highest_price:
            self.highest_price = current_price
            
            # Calculer nouveau stop
            if self.use_atr_trailing and self.atr.IsReady:
                new_stop = current_price - (self.atr_trailing_mult * self.atr.Current.Value)
            else:
                new_stop = current_price * (1 - self.trailing_percent)
            
            # Le stop ne peut que monter
            if new_stop > self.current_stop:
                old_stop = self.current_stop
                self.current_stop = new_stop
                self.Debug(f"{self.Time.date()}: Trail stop raised ${old_stop:.2f} -> ${new_stop:.2f}")
    
    def OnData(self, data):
        if self.IsWarmingUp:
            return
        
        if not data.ContainsKey(self.spy):
            return
        
        current_price = self.Securities[self.spy].Price
        
        # Entry logic
        if not self.Portfolio[self.spy].Invested:
            if current_price > self.sma.Current.Value:
                # Initialiser trailing stop
                self.entry_price = current_price
                self.highest_price = current_price
                
                if self.use_atr_trailing and self.atr.IsReady:
                    self.current_stop = current_price - (self.atr_trailing_mult * self.atr.Current.Value)
                else:
                    self.current_stop = current_price * (1 - self.trailing_percent)
                
                self.SetHoldings(self.spy, 0.95)
                
                self.Debug(f"{self.Time.date()}: BUY @ ${current_price:.2f}")
                self.Debug(f"  Initial trailing stop: ${self.current_stop:.2f}")
        
        # Manage trailing stop
        else:
            # Update trailing stop
            self.UpdateTrailingStop(current_price)
            
            # Check if stop hit
            if current_price <= self.current_stop:
                profit_pct = (current_price - self.entry_price) / self.entry_price
                self.Liquidate(self.spy)
                
                self.Debug(f"{self.Time.date()}: TRAILING STOP @ ${current_price:.2f}")
                self.Debug(f"  P/L: {profit_pct:.1%}")
                self.Debug(f"  Highest price reached: ${self.highest_price:.2f}")
                
                # Reset
                self.entry_price = None
                self.highest_price = None
                self.current_stop = None

print("TrailingStopAlgorithm defini")
print("\nFonctionnement:")
print("  1. Entree: Stop initial a 2x ATR sous le prix")
print("  2. Prix monte: Le stop monte aussi")
print("  3. Prix baisse: Le stop reste fixe")
print("  4. Stop touche: Position liquidee")

In [ ]:
# Visualisation du trailing stop
np.random.seed(123)

# Simuler un prix avec tendance haussiere puis retournement
n_days = 80
dates = pd.date_range('2023-01-01', periods=n_days, freq='B')

# Prix: monte puis descend
trend = np.concatenate([
    np.linspace(0, 25, 50),   # Forte hausse
    np.linspace(25, 10, 30),  # Correction
])
noise = np.random.normal(0, 1.5, n_days)
prices = 100 + trend + np.cumsum(noise * 0.3)

# Simuler trailing stop
entry_day = 5
entry_price = prices[entry_day]
trailing_pct = 0.05

trailing_stops = []
highest_prices = []
highest = entry_price
stop = entry_price * (1 - trailing_pct)
stop_triggered_day = None

for i, price in enumerate(prices):
    if i < entry_day:
        trailing_stops.append(np.nan)
        highest_prices.append(np.nan)
    else:
        if stop_triggered_day is None:
            if price > highest:
                highest = price
                stop = price * (1 - trailing_pct)
            
            if price <= stop:
                stop_triggered_day = i
            
            trailing_stops.append(stop)
            highest_prices.append(highest)
        else:
            trailing_stops.append(np.nan)
            highest_prices.append(np.nan)

# Graphique
plt.figure(figsize=(14, 6))

plt.plot(dates, prices, 'b-', linewidth=2, label='Prix')
plt.plot(dates, trailing_stops, 'r-', linewidth=2, label='Trailing Stop (5%)')
plt.plot(dates, highest_prices, 'g--', linewidth=1, alpha=0.7, label='Plus haut')

# Marquer entry et exit
plt.scatter([dates[entry_day]], [entry_price], color='green', s=150, marker='^', zorder=5, label=f'Entry: ${entry_price:.2f}')
if stop_triggered_day:
    exit_price = prices[stop_triggered_day]
    plt.scatter([dates[stop_triggered_day]], [exit_price], color='red', s=150, marker='v', zorder=5, label=f'Exit: ${exit_price:.2f}')
    
    # Zone de profit
    plt.fill_between(dates[entry_day:stop_triggered_day+1], 
                    entry_price, 
                    prices[entry_day:stop_triggered_day+1],
                    alpha=0.2, color='green', label='Profit zone')

plt.title('Trailing Stop en Action', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Prix ($)', fontsize=12)
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

if stop_triggered_day:
    profit = (exit_price - entry_price) / entry_price
    max_profit = (max(prices[entry_day:stop_triggered_day+1]) - entry_price) / entry_price
    print(f"\nResultat:")
    print(f"  Entry: ${entry_price:.2f}")
    print(f"  Exit: ${exit_price:.2f}")
    print(f"  Profit realise: {profit:.1%}")
    print(f"  Profit max possible: {max_profit:.1%}")
    print(f"  Profit capture: {profit/max_profit:.0%} du max")

---

## Partie 3 : Portfolio Heat et Exposure Limits (20 min)

### 3.1 Portfolio Heat (10 min)

Le **Portfolio Heat** represente le risque total du portefeuille : c'est la somme de tous les montants a risque sur toutes les positions ouvertes.

#### Formule

$$\text{Portfolio Heat} = \frac{\sum_{i} |\text{Entry}_i - \text{Stop}_i| \times \text{Quantity}_i}{\text{Total Portfolio Value}}$$

#### Regle de Van Tharp

> "Portfolio Heat should not exceed 20-25% at any time."

Cela signifie que meme si TOUS vos stops sont touches simultanement, vous ne perdez pas plus de 25% de votre capital.

In [ ]:
class PortfolioHeatAlgorithm(QCAlgorithm):
    """
    Algorithme qui surveille et limite le Portfolio Heat.
    Refuse d'ouvrir de nouvelles positions si le heat depasse le seuil.
    """
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Multiple actifs
        tickers = ["SPY", "QQQ", "IWM", "TLT", "GLD"]
        self.symbols = [self.AddEquity(t, Resolution.Daily).Symbol for t in tickers]
        
        # Indicateurs
        self.rsi = {s: self.RSI(s, 14) for s in self.symbols}
        
        # Stop tracking
        self.stops = {}  # symbol -> stop price
        self.stop_percent = 0.05  # 5% stop par position
        
        # Portfolio Heat limits
        self.max_portfolio_heat = 0.20  # Maximum 20% de heat
        self.risk_per_trade = 0.02      # 2% de risque par trade
        
        self.SetWarmup(30)
    
    def CalculatePortfolioHeat(self):
        """
        Calcule le Portfolio Heat total (risque cumule de toutes les positions).
        """
        total_risk = 0
        portfolio_value = self.Portfolio.TotalPortfolioValue
        
        for symbol in self.Portfolio.Keys:
            holding = self.Portfolio[symbol]
            
            if holding.Invested:
                entry = holding.AveragePrice
                stop = self.stops.get(symbol, entry * (1 - self.stop_percent))
                
                # Risque = distance au stop * quantite
                risk = abs(entry - stop) * abs(holding.Quantity)
                total_risk += risk
        
        heat = total_risk / portfolio_value if portfolio_value > 0 else 0
        return heat
    
    def CanAddPosition(self, new_risk_amount):
        """
        Verifie si on peut ajouter une nouvelle position sans depasser le heat max.
        """
        current_heat = self.CalculatePortfolioHeat()
        portfolio_value = self.Portfolio.TotalPortfolioValue
        new_heat = current_heat + (new_risk_amount / portfolio_value)
        
        return new_heat <= self.max_portfolio_heat
    
    def OnData(self, data):
        if self.IsWarmingUp:
            return
        
        # Log periodic heat
        if self.Time.day == 1:
            heat = self.CalculatePortfolioHeat()
            self.Debug(f"{self.Time.date()}: Portfolio Heat = {heat:.1%}")
        
        for symbol in self.symbols:
            if not data.ContainsKey(symbol):
                continue
            
            current_price = self.Securities[symbol].Price
            rsi = self.rsi[symbol]
            
            if not rsi.IsReady:
                continue
            
            # Entry signal: RSI oversold
            if not self.Portfolio[symbol].Invested and rsi.Current.Value < 30:
                # Calculer le risque de la nouvelle position
                portfolio_value = self.Portfolio.TotalPortfolioValue
                risk_amount = portfolio_value * self.risk_per_trade
                
                # Verifier si on peut ajouter cette position
                if self.CanAddPosition(risk_amount):
                    # Calculer position size
                    stop_price = current_price * (1 - self.stop_percent)
                    stop_distance = current_price - stop_price
                    shares = int(risk_amount / stop_distance)
                    
                    if shares > 0:
                        self.MarketOrder(symbol, shares)
                        self.stops[symbol] = stop_price
                        
                        new_heat = self.CalculatePortfolioHeat()
                        self.Debug(f"{self.Time.date()}: BUY {symbol.Value} - Heat now {new_heat:.1%}")
                else:
                    self.Debug(f"{self.Time.date()}: BLOCKED {symbol.Value} - Heat would exceed {self.max_portfolio_heat:.0%}")
            
            # Exit: RSI overbought or stop hit
            elif self.Portfolio[symbol].Invested:
                stop = self.stops.get(symbol, 0)
                
                if current_price <= stop or rsi.Current.Value > 70:
                    self.Liquidate(symbol)
                    if symbol in self.stops:
                        del self.stops[symbol]
                    
                    reason = "STOP" if current_price <= stop else "RSI>70"
                    new_heat = self.CalculatePortfolioHeat()
                    self.Debug(f"{self.Time.date()}: SELL {symbol.Value} ({reason}) - Heat now {new_heat:.1%}")

print("PortfolioHeatAlgorithm defini")
print("\nLimites:")
print("  - Max Portfolio Heat: 20%")
print("  - Risque par trade: 2%")
print("  - Stop par position: 5%")
print("\nMax positions simultanees: ~10 (20% / 2%)")

### Importance des Exposure Limits en Production

Les fonds institutionnels ont typiquement des limites strictes :

| Type de Fonds | Single Position | Sector | Gross |
|--------------|-----------------|--------|-------|
| **Hedge Fund agressif** | 15-20% | 40% | 200%+ |
| **Hedge Fund standard** | 5-10% | 25% | 150% |
| **Mutual Fund** | 5% | 25% | 100% |
| **Pension Fund** | 2-5% | 15% | 100% |

> **Conseil** : Meme si vous tradez un compte personnel, adoptez les pratiques institutionnelles. Les limites existent pour une raison : proteger le capital.

### 3.2 Exposure Limits (10 min)

Les **Exposure Limits** sont des contraintes supplementaires pour assurer la diversification :

| Limite | Description | Exemple |
|--------|-------------|--------|
| **Single Position** | Max par actif | Max 10% du portfolio |
| **Sector Exposure** | Max par secteur | Max 30% en Tech |
| **Gross Exposure** | Long + Short total | Max 150% |
| **Net Exposure** | Long - Short | Entre -20% et +100% |

In [ ]:
class ExposureLimitsAlgorithm(QCAlgorithm):
    """
    Algorithme avec limites d'exposition strictes.
    """
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Actifs par secteur
        self.sectors = {
            "Tech": ["AAPL", "MSFT", "GOOGL", "NVDA"],
            "Finance": ["JPM", "BAC", "GS"],
            "Healthcare": ["JNJ", "PFE", "UNH"],
            "Energy": ["XOM", "CVX"],
        }
        
        # Ajouter tous les actifs
        self.symbols = []
        self.symbol_to_sector = {}
        
        for sector, tickers in self.sectors.items():
            for ticker in tickers:
                symbol = self.AddEquity(ticker, Resolution.Daily).Symbol
                self.symbols.append(symbol)
                self.symbol_to_sector[symbol] = sector
        
        # Exposure limits
        self.max_single_position = 0.10    # Max 10% par actif
        self.max_sector_exposure = 0.30    # Max 30% par secteur
        self.max_gross_exposure = 1.00     # Max 100% gross (pas de leverage)
        self.max_long_exposure = 1.00      # Max 100% long
        
        self.SetWarmup(30)
    
    def GetCurrentExposures(self):
        """
        Calcule les expositions actuelles.
        """
        portfolio_value = self.Portfolio.TotalPortfolioValue
        
        # Exposition par actif
        position_weights = {}
        for symbol in self.symbols:
            value = self.Portfolio[symbol].HoldingsValue
            position_weights[symbol] = value / portfolio_value if portfolio_value > 0 else 0
        
        # Exposition par secteur
        sector_weights = {sector: 0 for sector in self.sectors.keys()}
        for symbol, weight in position_weights.items():
            sector = self.symbol_to_sector.get(symbol)
            if sector:
                sector_weights[sector] += weight
        
        # Expositions totales
        long_exposure = sum(max(0, w) for w in position_weights.values())
        short_exposure = sum(abs(min(0, w)) for w in position_weights.values())
        gross_exposure = long_exposure + short_exposure
        net_exposure = long_exposure - short_exposure
        
        return {
            "positions": position_weights,
            "sectors": sector_weights,
            "long": long_exposure,
            "short": short_exposure,
            "gross": gross_exposure,
            "net": net_exposure,
        }
    
    def CanEnterPosition(self, symbol, proposed_weight):
        """
        Verifie si on peut entrer la position sans violer les limites.
        """
        exposures = self.GetCurrentExposures()
        sector = self.symbol_to_sector.get(symbol)
        
        # Check single position limit
        current_weight = exposures["positions"].get(symbol, 0)
        new_weight = current_weight + proposed_weight
        
        if abs(new_weight) > self.max_single_position:
            self.Debug(f"BLOCKED: {symbol.Value} would exceed single position limit ({new_weight:.1%} > {self.max_single_position:.0%})")
            return False
        
        # Check sector limit
        if sector:
            current_sector = exposures["sectors"].get(sector, 0)
            new_sector = current_sector + proposed_weight
            
            if new_sector > self.max_sector_exposure:
                self.Debug(f"BLOCKED: {symbol.Value} would exceed {sector} sector limit ({new_sector:.1%} > {self.max_sector_exposure:.0%})")
                return False
        
        # Check gross exposure
        new_gross = exposures["gross"] + abs(proposed_weight)
        if new_gross > self.max_gross_exposure:
            self.Debug(f"BLOCKED: {symbol.Value} would exceed gross exposure ({new_gross:.1%} > {self.max_gross_exposure:.0%})")
            return False
        
        return True
    
    def OnData(self, data):
        if self.IsWarmingUp:
            return
        
        # Log exposures weekly
        if self.Time.weekday() == 0:  # Monday
            exp = self.GetCurrentExposures()
            self.Debug(f"\n{self.Time.date()}: EXPOSURE REPORT")
            self.Debug(f"  Gross: {exp['gross']:.1%}, Net: {exp['net']:.1%}")
            for sector, weight in exp['sectors'].items():
                if weight > 0:
                    self.Debug(f"  {sector}: {weight:.1%}")
        
        # Simple allocation logic (equal weight with limits)
        target_weight = 0.08  # 8% par actif
        
        for symbol in self.symbols:
            if not data.ContainsKey(symbol):
                continue
            
            current_weight = self.Portfolio[symbol].HoldingsValue / self.Portfolio.TotalPortfolioValue
            
            # Rebalance if deviation > 2%
            if abs(current_weight - target_weight) > 0.02:
                delta_weight = target_weight - current_weight
                
                if self.CanEnterPosition(symbol, delta_weight):
                    self.SetHoldings(symbol, target_weight)

print("ExposureLimitsAlgorithm defini")
print("\nLimites d'exposition:")
print("  - Single Position: max 10%")
print("  - Sector Exposure: max 30%")
print("  - Gross Exposure: max 100%")

---

## Partie 4 : Drawdown Control et Volatility Scaling (20 min)

### 4.1 Max Drawdown Monitor (10 min)

Le **Drawdown** est la perte depuis le plus haut historique du portefeuille. C'est une metrique cruciale pour evaluer le risque.

$$\text{Drawdown} = \frac{\text{Peak Value} - \text{Current Value}}{\text{Peak Value}}$$

#### Circuit Breakers

Beaucoup de fonds implementent des "circuit breakers" qui reduisent ou ferment les positions quand le drawdown depasse un seuil.

In [ ]:
class DrawdownControlAlgorithm(QCAlgorithm):
    """
    Algorithme avec controle de drawdown.
    Liquide toutes les positions si le drawdown depasse le seuil.
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        self.qqq = self.AddEquity("QQQ", Resolution.Daily).Symbol
        
        # Drawdown parameters
        self.max_allowed_drawdown = 0.15  # 15% max drawdown
        self.peak_value = self.Portfolio.TotalPortfolioValue
        self.trading_halted = False
        self.halt_recovery_threshold = 0.05  # Resume trading if drawdown < 5%
        
        # Indicateurs
        self.sma_spy = self.SMA(self.spy, 50)
        self.sma_qqq = self.SMA(self.qqq, 50)
        
        self.SetWarmup(50)
    
    def MonitorDrawdown(self):
        """
        Surveille le drawdown et declenche la liquidation si necessaire.
        """
        current_value = self.Portfolio.TotalPortfolioValue
        
        # Update peak
        if current_value > self.peak_value:
            self.peak_value = current_value
        
        # Calculate drawdown
        drawdown = (self.peak_value - current_value) / self.peak_value
        
        # Check if we should halt trading
        if drawdown > self.max_allowed_drawdown and not self.trading_halted:
            self.Liquidate()
            self.trading_halted = True
            self.Debug(f"{self.Time.date()}: *** MAX DRAWDOWN EXCEEDED ***")
            self.Debug(f"  Drawdown: {drawdown:.1%}")
            self.Debug(f"  Peak: ${self.peak_value:,.0f}")
            self.Debug(f"  Current: ${current_value:,.0f}")
            self.Debug("  All positions liquidated. Trading halted.")
        
        # Check if we can resume trading
        elif self.trading_halted and drawdown < self.halt_recovery_threshold:
            self.trading_halted = False
            # Reset peak to current value
            self.peak_value = current_value
            self.Debug(f"{self.Time.date()}: Trading resumed. Drawdown recovered to {drawdown:.1%}")
        
        return drawdown
    
    def OnData(self, data):
        if self.IsWarmingUp:
            return
        
        # Monitor drawdown first
        drawdown = self.MonitorDrawdown()
        
        # Don't trade if halted
        if self.trading_halted:
            return
        
        # Simple trend following strategy
        for symbol, sma in [(self.spy, self.sma_spy), (self.qqq, self.sma_qqq)]:
            if not data.ContainsKey(symbol):
                continue
            
            price = self.Securities[symbol].Price
            
            if price > sma.Current.Value:
                if not self.Portfolio[symbol].Invested:
                    self.SetHoldings(symbol, 0.45)
                    self.Debug(f"{self.Time.date()}: BUY {symbol.Value} (DD: {drawdown:.1%})")
            else:
                if self.Portfolio[symbol].Invested:
                    self.Liquidate(symbol)
                    self.Debug(f"{self.Time.date()}: SELL {symbol.Value}")

print("DrawdownControlAlgorithm defini")
print("\nParametres:")
print("  - Max drawdown: 15%")
print("  - Recovery threshold: 5%")
print("\nFonctionnement:")
print("  1. Drawdown > 15%: Liquidation totale, trading arrete")
print("  2. Drawdown < 5%: Trading reprend")

### 4.2 Volatility Scaling (10 min)

Le **Volatility Scaling** ajuste dynamiquement l'exposition du portefeuille en fonction de la volatilite du marche.

#### Principe

- Volatilite haute -> Reduire l'exposition
- Volatilite basse -> Augmenter l'exposition

#### Formule

$$\text{Adjusted Weight} = \text{Base Weight} \times \frac{\text{Target Vol}}{\text{Current Vol}}$$

Avec un cap (ex: 150%) pour eviter le sur-leveraging quand la vol est tres basse.

In [ ]:
class VolatilityScalingAlgorithm(QCAlgorithm):
    """
    Algorithme qui ajuste l'exposition selon la volatilite.
    Reduit l'exposition quand la vol augmente.
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        
        # Indicateur de volatilite (utiliser realized vol sur 20 jours)
        # Note: En production, on pourrait utiliser VIX
        self.vol_window = 20
        self.returns_history = []
        
        # Volatility scaling parameters
        self.target_volatility = 0.15  # Cible: 15% vol annualisee
        self.base_weight = 1.0         # Poids de base: 100%
        self.max_scalar = 1.5          # Max 150% d'exposition
        self.min_scalar = 0.25         # Min 25% d'exposition
        
        # Indicateurs
        self.sma = self.SMA(self.spy, 200)
        
        # Track previous price for returns
        self.previous_price = None
        
        self.SetWarmup(200)
    
    def CalculateRealizedVol(self):
        """
        Calcule la volatilite realisee annualisee.
        """
        if len(self.returns_history) < self.vol_window:
            return self.target_volatility  # Default to target
        
        recent_returns = self.returns_history[-self.vol_window:]
        daily_vol = np.std(recent_returns)
        annual_vol = daily_vol * np.sqrt(252)
        
        return annual_vol
    
    def VolatilityAdjustedWeight(self):
        """
        Calcule le poids ajuste par la volatilite.
        """
        current_vol = self.CalculateRealizedVol()
        
        if current_vol <= 0:
            return self.base_weight
        
        # Volatility scalar
        vol_scalar = self.target_volatility / current_vol
        
        # Cap the scalar
        vol_scalar = max(self.min_scalar, min(vol_scalar, self.max_scalar))
        
        # Adjusted weight
        adjusted_weight = self.base_weight * vol_scalar
        
        return adjusted_weight, current_vol, vol_scalar
    
    def OnData(self, data):
        if not data.ContainsKey(self.spy):
            return
        
        current_price = self.Securities[self.spy].Price
        
        # Update returns history
        if self.previous_price is not None:
            daily_return = (current_price - self.previous_price) / self.previous_price
            self.returns_history.append(daily_return)
            
            # Keep only recent history
            if len(self.returns_history) > 100:
                self.returns_history = self.returns_history[-100:]
        
        self.previous_price = current_price
        
        if self.IsWarmingUp:
            return
        
        # Calculate volatility-adjusted weight
        adjusted_weight, current_vol, vol_scalar = self.VolatilityAdjustedWeight()
        
        # Log weekly
        if self.Time.weekday() == 0:
            self.Debug(f"{self.Time.date()}: Vol={current_vol:.1%}, Scalar={vol_scalar:.2f}, Weight={adjusted_weight:.0%}")
        
        # Simple trend strategy with vol-adjusted sizing
        if current_price > self.sma.Current.Value:
            # Bullish: invest with vol-adjusted weight
            target = min(adjusted_weight, 1.0)  # Cap at 100% (no leverage)
            
            current_weight = self.Portfolio[self.spy].HoldingsValue / self.Portfolio.TotalPortfolioValue
            
            # Rebalance if deviation > 5%
            if abs(current_weight - target) > 0.05:
                self.SetHoldings(self.spy, target)
        else:
            # Bearish: exit
            if self.Portfolio[self.spy].Invested:
                self.Liquidate(self.spy)

print("VolatilityScalingAlgorithm defini")
print("\nParametres:")
print("  - Target volatility: 15%")
print("  - Max scalar: 1.5x (150%)")
print("  - Min scalar: 0.25x (25%)")
print("\nExemples:")
print("  - Vol = 30% -> Scalar = 0.5 -> Weight = 50%")
print("  - Vol = 15% -> Scalar = 1.0 -> Weight = 100%")
print("  - Vol = 10% -> Scalar = 1.5 -> Weight = 150% (capped at 100%)")

---

## Partie 5 : Risk Models QuantConnect (15 min)

QuantConnect fournit des **Risk Models** pre-implementes qu'on peut integrer facilement.

### Risk Models Disponibles

| Model | Description |
|-------|-------------|
| `MaximumDrawdownPercentPerSecurity` | Liquide si drawdown > X% par actif |
| `MaximumUnrealizedProfitPercentPerSecurity` | Prend profit si gain > X% |
| `MaximumSectorExposureRiskManagementModel` | Limite l'exposition par secteur |
| `TrailingStopRiskManagementModel` | Trailing stop automatique |

### Composite Risk Models

QuantConnect permet de combiner plusieurs Risk Models dans un `CompositeRiskManagementModel` :

```python
from AlgorithmImports import *

# Exemple de Risk Model composite
risk_model = CompositeRiskManagementModel(
    MaximumDrawdownPercentPerSecurity(0.05),          # Stop-loss 5%
    MaximumUnrealizedProfitPercentPerSecurity(0.15),  # Take profit 15%
    MaximumSectorExposureRiskManagementModel(0.30),   # Max 30% par secteur
)
self.SetRiskManagement(risk_model)
```

Cette approche permet d'avoir plusieurs niveaux de protection simultanement.

In [ ]:
class BuiltInRiskModelsAlgorithm(QCAlgorithm):
    """
    Demonstration des Risk Models natifs de QuantConnect.
    """
    
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # Ajouter actifs
        self.spy = self.AddEquity("SPY", Resolution.Daily).Symbol
        self.qqq = self.AddEquity("QQQ", Resolution.Daily).Symbol
        self.tlt = self.AddEquity("TLT", Resolution.Daily).Symbol
        
        # === OPTION 1: Max Drawdown per Security ===
        # Liquide automatiquement si une position perd plus de 5%
        self.SetRiskManagement(MaximumDrawdownPercentPerSecurity(0.05))
        
        # === OPTION 2: Trailing Stop (alternative) ===
        # self.SetRiskManagement(TrailingStopRiskManagementModel(0.03))  # 3% trail
        
        # === OPTION 3: Multiple Risk Models (composite) ===
        # risk_model = CompositeRiskManagementModel(
        #     MaximumDrawdownPercentPerSecurity(0.05),
        #     MaximumUnrealizedProfitPercentPerSecurity(0.10),  # Take profit at 10%
        # )
        # self.SetRiskManagement(risk_model)
        
        # Indicateurs
        self.sma_spy = self.SMA(self.spy, 50)
        
        self.SetWarmup(50)
    
    def OnData(self, data):
        if self.IsWarmingUp:
            return
        
        # Simple allocation strategy
        # Le Risk Model s'occupera automatiquement des stops
        if not self.Portfolio[self.spy].Invested:
            price = self.Securities[self.spy].Price
            if price > self.sma_spy.Current.Value:
                self.SetHoldings(self.spy, 0.4)
                self.SetHoldings(self.qqq, 0.3)
                self.SetHoldings(self.tlt, 0.2)
                self.Debug(f"{self.Time.date()}: Positions opened - Risk Model will manage stops")

print("BuiltInRiskModelsAlgorithm defini")
print("\nRisk Models utilises:")
print("  - MaximumDrawdownPercentPerSecurity(0.05)")
print("  - Liquide automatiquement si perte > 5% sur une position")
print("\nAvantage: Code simplifie, risk management automatique")

In [ ]:
# Liste complete des Risk Models QuantConnect
print("Risk Models QuantConnect Disponibles")
print("=" * 60)

risk_models = [
    {
        "name": "MaximumDrawdownPercentPerSecurity",
        "usage": "MaximumDrawdownPercentPerSecurity(0.05)",
        "description": "Liquide si drawdown > X% par position"
    },
    {
        "name": "MaximumDrawdownPercentPortfolio",
        "usage": "MaximumDrawdownPercentPortfolio(0.10)",
        "description": "Liquide TOUT si drawdown portfolio > X%"
    },
    {
        "name": "MaximumUnrealizedProfitPercentPerSecurity",
        "usage": "MaximumUnrealizedProfitPercentPerSecurity(0.10)",
        "description": "Take profit si gain > X% par position"
    },
    {
        "name": "TrailingStopRiskManagementModel",
        "usage": "TrailingStopRiskManagementModel(0.03)",
        "description": "Trailing stop automatique a X%"
    },
    {
        "name": "MaximumSectorExposureRiskManagementModel",
        "usage": "MaximumSectorExposureRiskManagementModel(0.25)",
        "description": "Limite exposition par secteur a X%"
    },
    {
        "name": "NullRiskManagementModel",
        "usage": "NullRiskManagementModel()",
        "description": "Desactive le risk management (defaut)"
    },
]

for model in risk_models:
    print(f"\n{model['name']}")
    print(f"  Usage: self.SetRiskManagement({model['usage']})")
    print(f"  Description: {model['description']}")

---

## Partie 6 : Strategie Complete avec Risk Management (20 min)

Combinons tous les elements vus dans ce notebook dans une strategie complete.

In [ ]:
class ComprehensiveRiskManagedStrategy(QCAlgorithm):
    """
    Strategie complete avec tous les elements de risk management:
    
    1. Position Sizing: 1% risk per trade (Fixed Fractional)
    2. Stop-Loss: 2x ATR
    3. Trailing Stop: Active apres +10%
    4. Portfolio Heat: Max 6%
    5. Max Drawdown: 15%
    6. Exposure Limits: Max 20% par position
    """
    
    def Initialize(self):
        self.SetStartDate(2022, 1, 1)
        self.SetEndDate(2023, 12, 31)
        self.SetCash(100000)
        
        # === ACTIFS ===
        tickers = ["SPY", "QQQ", "IWM", "TLT", "GLD"]
        self.symbols = []
        self.atr = {}
        self.rsi = {}
        
        for ticker in tickers:
            symbol = self.AddEquity(ticker, Resolution.Daily).Symbol
            self.symbols.append(symbol)
            self.atr[symbol] = self.ATR(symbol, 14)
            self.rsi[symbol] = self.RSI(symbol, 14)
        
        # === RISK PARAMETERS ===
        # Position sizing
        self.risk_per_trade = 0.01        # 1% risk per trade
        self.atr_stop_multiplier = 2.0    # Stop at 2x ATR
        
        # Portfolio limits
        self.max_portfolio_heat = 0.06    # Max 6% total heat
        self.max_single_position = 0.20   # Max 20% per position
        self.max_positions = 5            # Max 5 positions
        
        # Drawdown
        self.max_drawdown = 0.15          # 15% max drawdown
        self.peak_value = self.Portfolio.TotalPortfolioValue
        self.trading_halted = False
        
        # Trailing stop
        self.trailing_activation = 0.10    # Activate trail after +10%
        self.trailing_distance = 0.05      # 5% trailing distance
        
        # === POSITION TRACKING ===
        self.positions = {}  # symbol -> {entry, stop, highest, trailing_active}
        
        self.SetWarmup(30)
    
    # === POSITION SIZING ===
    def CalculatePositionSize(self, symbol, entry_price):
        """
        Fixed Fractional avec stop ATR.
        """
        portfolio_value = self.Portfolio.TotalPortfolioValue
        risk_amount = portfolio_value * self.risk_per_trade
        
        # ATR-based stop
        atr_value = self.atr[symbol].Current.Value
        stop_distance = self.atr_stop_multiplier * atr_value
        stop_price = entry_price - stop_distance
        
        if stop_distance <= 0:
            return 0, 0
        
        # Calculate shares
        shares = int(risk_amount / stop_distance)
        
        # Apply position limit
        max_shares = int((portfolio_value * self.max_single_position) / entry_price)
        shares = min(shares, max_shares)
        
        return shares, stop_price
    
    # === PORTFOLIO HEAT ===
    def CalculatePortfolioHeat(self):
        """
        Total risk across all positions.
        """
        total_risk = 0
        portfolio_value = self.Portfolio.TotalPortfolioValue
        
        for symbol, pos in self.positions.items():
            if self.Portfolio[symbol].Invested:
                holding = self.Portfolio[symbol]
                risk = abs(holding.AveragePrice - pos['stop']) * abs(holding.Quantity)
                total_risk += risk
        
        return total_risk / portfolio_value if portfolio_value > 0 else 0
    
    # === DRAWDOWN MONITOR ===
    def CheckDrawdown(self):
        """
        Monitor and react to drawdown.
        """
        current_value = self.Portfolio.TotalPortfolioValue
        
        if current_value > self.peak_value:
            self.peak_value = current_value
        
        drawdown = (self.peak_value - current_value) / self.peak_value
        
        if drawdown > self.max_drawdown and not self.trading_halted:
            self.Liquidate()
            self.positions.clear()
            self.trading_halted = True
            self.Debug(f"{self.Time.date()}: *** DRAWDOWN CIRCUIT BREAKER *** ({drawdown:.1%})")
        
        elif self.trading_halted and drawdown < 0.05:
            self.trading_halted = False
            self.peak_value = current_value
            self.Debug(f"{self.Time.date()}: Trading resumed")
        
        return drawdown
    
    # === TRAILING STOP ===
    def UpdateTrailingStops(self):
        """
        Update trailing stops for all positions.
        """
        for symbol, pos in list(self.positions.items()):
            if not self.Portfolio[symbol].Invested:
                continue
            
            current_price = self.Securities[symbol].Price
            entry_price = pos['entry']
            profit_pct = (current_price - entry_price) / entry_price
            
            # Activate trailing stop after +10%
            if profit_pct >= self.trailing_activation and not pos['trailing_active']:
                pos['trailing_active'] = True
                self.Debug(f"{self.Time.date()}: {symbol.Value} - Trailing stop ACTIVATED at {profit_pct:.1%}")
            
            # Update trailing stop if active
            if pos['trailing_active']:
                if current_price > pos['highest']:
                    pos['highest'] = current_price
                    new_stop = current_price * (1 - self.trailing_distance)
                    if new_stop > pos['stop']:
                        pos['stop'] = new_stop
    
    # === MAIN TRADING LOGIC ===
    def OnData(self, data):
        if self.IsWarmingUp:
            return
        
        # 1. Check drawdown first
        drawdown = self.CheckDrawdown()
        if self.trading_halted:
            return
        
        # 2. Update trailing stops
        self.UpdateTrailingStops()
        
        # 3. Check stop-losses
        for symbol in list(self.positions.keys()):
            if not data.ContainsKey(symbol):
                continue
            
            current_price = self.Securities[symbol].Price
            pos = self.positions[symbol]
            
            if current_price <= pos['stop']:
                self.Liquidate(symbol)
                pnl = (current_price - pos['entry']) / pos['entry']
                stop_type = "TRAILING" if pos['trailing_active'] else "INITIAL"
                self.Debug(f"{self.Time.date()}: {symbol.Value} - {stop_type} STOP @ ${current_price:.2f} (P/L: {pnl:.1%})")
                del self.positions[symbol]
        
        # 4. Look for new entries
        current_heat = self.CalculatePortfolioHeat()
        num_positions = len(self.positions)
        
        for symbol in self.symbols:
            if symbol in self.positions:
                continue
            
            if not data.ContainsKey(symbol):
                continue
            
            # Check limits
            if num_positions >= self.max_positions:
                break
            
            if current_heat >= self.max_portfolio_heat - self.risk_per_trade:
                break
            
            # Entry signal: RSI oversold
            rsi = self.rsi[symbol]
            if not rsi.IsReady or not self.atr[symbol].IsReady:
                continue
            
            if rsi.Current.Value < 30:
                current_price = self.Securities[symbol].Price
                shares, stop_price = self.CalculatePositionSize(symbol, current_price)
                
                if shares > 0:
                    self.MarketOrder(symbol, shares)
                    
                    self.positions[symbol] = {
                        'entry': current_price,
                        'stop': stop_price,
                        'highest': current_price,
                        'trailing_active': False
                    }
                    
                    num_positions += 1
                    current_heat = self.CalculatePortfolioHeat()
                    
                    self.Debug(f"{self.Time.date()}: BUY {symbol.Value} - {shares} shares @ ${current_price:.2f}")
                    self.Debug(f"  Stop: ${stop_price:.2f}, Heat: {current_heat:.1%}")
        
        # 5. Log weekly status
        if self.Time.weekday() == 0:
            self.Debug(f"\n{self.Time.date()}: Weekly Status")
            self.Debug(f"  Portfolio: ${self.Portfolio.TotalPortfolioValue:,.0f}")
            self.Debug(f"  Drawdown: {drawdown:.1%}")
            self.Debug(f"  Heat: {current_heat:.1%}")
            self.Debug(f"  Positions: {num_positions}")
    
    def OnEndOfAlgorithm(self):
        """Final summary."""
        self.Debug("\n" + "="*60)
        self.Debug("FINAL SUMMARY - Comprehensive Risk-Managed Strategy")
        self.Debug("="*60)
        self.Debug(f"Final Value: ${self.Portfolio.TotalPortfolioValue:,.0f}")
        self.Debug(f"Total Return: {(self.Portfolio.TotalPortfolioValue / 100000 - 1) * 100:.2f}%")

print("ComprehensiveRiskManagedStrategy defini")
print("\n" + "="*60)
print("RESUME DES PARAMETRES DE RISK MANAGEMENT")
print("="*60)
print("\n1. Position Sizing:")
print("   - Methode: Fixed Fractional")
print("   - Risque par trade: 1%")
print("   - Max par position: 20%")
print("\n2. Stop-Loss:")
print("   - Initial: 2x ATR")
print("   - Trailing: Active apres +10%, trail de 5%")
print("\n3. Portfolio Limits:")
print("   - Max Heat: 6%")
print("   - Max Positions: 5")
print("\n4. Drawdown Control:")
print("   - Max Drawdown: 15%")
print("   - Circuit breaker: Liquidation totale si depasse")

---

## Conclusion et Prochaines Etapes

### Recapitulatif

Dans ce notebook, nous avons couvert :

| Sujet | Points Cles |
|-------|-------------|
| **Position Sizing** | Fixed Fractional (1-2% risk), Kelly Criterion (optimal mais agressif), Half-Kelly recommande |
| **Stop-Loss** | Fixed % (simple), ATR-based (adaptatif), Trailing (protege gains) |
| **Portfolio Heat** | Somme des risques, max 20-25% recommande |
| **Exposure Limits** | Single position, sector, gross/net exposure |
| **Drawdown Control** | Circuit breakers, liquidation automatique |
| **Volatility Scaling** | Reduire exposition quand vol augmente |
| **Risk Models QC** | MaximumDrawdownPercentPerSecurity, TrailingStop, etc. |

### Regles d'Or du Risk Management

1. **Never risk more than 1-2% per trade**
2. **Keep portfolio heat under 20-25%**
3. **Have a max drawdown limit (15-25%)**
4. **Use trailing stops to protect gains**
5. **Scale position size with account equity**
6. **Reduce exposure when volatility spikes**

### Prochaines Etapes

Dans les notebooks suivants, nous couvrirons :

- **QC-Py-11**: Backtesting avance et analyse de performance
- **QC-Py-12**: Optimisation de parametres et walk-forward analysis
- **QC-Py-13**: Machine Learning pour le trading

### Ressources Complementaires

- [Van K. Tharp - Trade Your Way to Financial Freedom](https://www.amazon.com/Trade-Your-Way-Financial-Freedom/dp/007147871X)
- [QuantConnect Risk Management Documentation](https://www.quantconnect.com/docs/v2/writing-algorithms/algorithm-framework/risk-management)
- [Kelly Criterion (Wikipedia)](https://en.wikipedia.org/wiki/Kelly_criterion)
- [ATR Indicator (Investopedia)](https://www.investopedia.com/terms/a/atr.asp)

---

**Notebook complete. Pret pour QC-Py-11.**